<a href="https://colab.research.google.com/github/honghusldl/capstone-project-city-segmentation/blob/main/Introduction_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction:

The aim of this project is to find the best neighborhood in Toronto to open a new Chinese restaurant. In this project, we will utilize FourSquare API to retrieve location data and also explore venues in Toronto. We will analyze the neighborhoods by two types of clustering. Type I is clustering neighborhoods by speicific one venue category, Chinese restaurant, whereas Type II is clustering neighborhoods by all venue cateogories. Considering business people may make their decisions from different angle, I provide two means here. Type I clustering (Chinese restaurant only) focuses on competitive strength from Chinese restaurant specifically, while Type II clustering (all venues) emphasizes competition between all businesses.

## Data Description:

1: Basic Toronto data including Toronto's borough, neighborhoods, postal codes data from Wikipedia.
Data sources: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

2: Geospatial data for Toronto's postal codes.
Data sources:https://cocl.us/Geospatial_data. Data is retrieved by Geocoder package.

3: Toronto's venues data including venue name, venue geospatial information (latitude and longitude) and venue categories.
Data is retrieved by FourSquare API with personal credentials.